In [5]:
from scipy.stats import norm
import numpy as np
import yfinance as yf
import pandas as pd
from ta.volatility import AverageTrueRange
from datetime import datetime


def get_historical_data(stk):
    df = yf.download(stk,period='max', interval="1wk").dropna()
    df['year']=df.index.isocalendar().year.astype(int)
    return df


def trade_parameters(df, yield_perc):
    df["strike_price"] = df.Open.round(0)
    df["straddle_yield"] = yield_perc
    df["straddle_premium"] = df.strike_price * yield_perc
    
    df['ATR']=AverageTrueRange(high=df.High,low=df.Low,close=df.Close,window=5,fillna=False).average_true_range().rank(pct=True).shift(1)
    df['abs_change']=abs(df.Close-df.Open)
    df['abs_change_pct']=df.abs_change/df.Open
    df["profit_loss"] = df.straddle_premium - df.abs_change
    df["worst"] = df.straddle_premium - df.abs_change
    df["log_return"] = np.log((df.strike_price + df.profit_loss) / df.strike_price)
    df['pos_ret']=np.where(df.log_return>0,df.log_return,0)
    df['neg_ret']=np.where(df.log_return<0,df.log_return,0)
    df = df.iloc[1: , ::]
    
    lim=0.85
    df=df[df.ATR<lim]
    
    return df

def annual_group(df):
    df = df.groupby(["year"]).agg(
        {"log_return": "sum",'pos_ret':'sum','neg_ret':'sum','profit_loss':'mean','worst':'min'}
    )
    df['sharpe']=df.pos_ret/abs(df.neg_ret)
    df['recovery_periods']=(-df.worst/df.profit_loss)
    df['recovery_periods']=df['recovery_periods'].mask(df['recovery_periods']<0,0)
    return(df)
    
    
atm_option_data = pd.read_csv("historical_straddle_data/option_data_2022-09-02_2022-08-29.csv")

#print(blank)
arr = []
limited=['FEZ']
atm_option_data['yield_perc']=(atm_option_data.atm_call_last + atm_option_data.atm_put_last) / atm_option_data.strike
atm_option_data=atm_option_data[atm_option_data.symbol.isin(limited)]
for i,j in zip(atm_option_data.symbol,atm_option_data.yield_perc):
    
    stock = i
    yield_perc = j
    df = get_historical_data(stock)
    #print(df.tail)
    df = trade_parameters(df, yield_perc)
    #print(df.tail)
    #df.to_csv(stock + ".csv")
    df = annual_group(df)

    #df=df.iloc[::,[0,3,4]]
    #print(df)
    
    years=np.arange(df.index[0], 2023, 1, dtype=int)
    zeros=np.zeros((len(years),len(df.columns)),dtype=float)
    blank=pd.DataFrame(columns=df.columns.values,index=years,data=zeros)

    df=df.replace(float('inf'),500)
    df.to_csv(stock + ".csv")
    print(j)
    print(df)
    lr_avg=(min(
        np.average(df.log_return),
        np.percentile(df.sharpe,50)
    ))

    sharpe_avg=(min(
        np.percentile(df.sharpe,50),
          np.average(df.sharpe)
    ))
    lr_check=len(df.log_return[df.log_return>0])/len(df.log_return)
    sharpe_check=len(df.sharpe[df.sharpe>2])/len(df.sharpe)
    recovery_check=len(df.recovery_periods[df.recovery_periods<7])/len(df.recovery_periods)
    lr_sum=np.sum(df.log_return)
    
    df1=blank.add(df,fill_value=0)
    periods_traded=len(df1.log_return[df1.log_return != 0])/len(df1.log_return)
    
    out=[stock,lr_avg,sharpe_avg,lr_sum,lr_check,sharpe_check,recovery_check,periods_traded]

    arr.append(out)
    #print(arr)
    df.to_csv('out.csv')
finaldf=pd.DataFrame(data=arr,columns=['stock','lr_avg','sharpe_avg','lr_sum','lr_check','sharpe_check','recovery_check','periods_traded'])
print(finaldf)
finaldf.to_csv('weekly_mass_bt.csv')

[*********************100%***********************]  1 of 1 completed
0.033823529411764704
      log_return   pos_ret   neg_ret  profit_loss     worst      sharpe  \
year                                                                      
2002    0.079983  0.112056 -0.032073     0.234380 -0.528235    3.493804   
2003    0.487719  0.698616 -0.210897     0.290192 -1.495884    3.312598   
2004    0.939597  0.981161 -0.041564     0.631543 -0.718531   23.605843   
2005    1.062634  1.065314 -0.002680     0.829865 -0.107058  397.500027   
2006    0.825013  0.897555 -0.072542     0.772992 -1.126471   12.372950   
2007    0.542820  0.638450 -0.095630     0.926765 -2.332941    6.676266   
2008    0.085307  0.185841 -0.100534     0.279118 -2.682940    1.848534   
2009    0.127245  0.298424 -0.171179     0.247979 -1.585589    1.743345   
2010    0.083445  0.454607 -0.371162     0.113252 -3.848530    1.224821   
2011    0.132677  0.317670 -0.184993     0.232851 -2.256175    1.717200   
2012    0.

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind